In [1]:
# test full training of maskcnn_polished_with_rcnn_k_bl
from os.path import join

import numpy as np

import torch

from thesis_v2 import dir_dict

from thesis_v2.data.prepared.cadena_plos_cb19 import get_data

from thesis_v2.training_extra.maskcnn_like.opt import get_maskcnn_v1_opt_config
from thesis_v2.training_extra.maskcnn_like.training import (train_one,
                                                            partial)

from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import (
    gen_maskcnn_polished_with_rcnn_k_bl,
    load_modules
)

In [2]:
def pad_nan(y_this):
    pad = np.full_like(y_this, fill_value=np.nan)
    return np.concatenate([pad, y_this, pad], axis=1)

def train_one_maskcnn_polished_with_rcnn_k_bl(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    n_timesteps,
):
    
    load_modules()
    datasets = get_data(
         px_kept=80, final_size=input_size, scale=0.5,
        seed=split_seed
    )

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': pad_nan(datasets[1]),
        'X_val': datasets[2].astype(np.float32),
        'y_val': pad_nan(datasets[3]),
        'X_test': datasets[4].astype(np.float32),
        'y_test': pad_nan(datasets[5]),
    }

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished_with_rcnn_k_bl(
                                    input_size=input_size_cnn,
                                    num_neuron=n,
                                    out_channel=out_channel,
                                    kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
                                    kernel_size_l23=3,
                                    act_fn=act_fn,
                                    pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
                                    pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
                                    num_layer=num_layer,
                                    n_timesteps=n_timesteps,
                                    factored_constraint=None,
                                    blstack_pool_ksize=1,
                                    blstack_pool_type=None,
                                    acc_mode='cummean',
                                    bn_after_fc=False,
                                    ff_1st_block=True,
                                    )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )
    
    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets,
        key=f'debug/test_full_training_maskcnn_polished_with_rcnn_k_bl/ff_1st_block_cb19_data-handle_nan-nan_y/{model_seed}',
        show_every=100,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False,
        extra_params={
            # reduce on batch axis
            'eval_fn': {
                'yhat_reduce_axis': 1,
                'handle_nan': True,
            }
        },
        print_model=True,
        handle_nan=True,
    )
    
    return result['stats_best']['stats']['test']['corr_mean'], result['stats_best']['stats']['test']['corr']

In [3]:
maskcnn_param_template = {
    'out_channel': 16,
    'num_layer': 3,
    'kernel_size_l1': 9,
    'pooling_ksize': 3,
    'pooling_type': 'avg',
    'model_seed': 0,
    'split_seed': 0,
}

maskcnn_param_regular = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'loss_type': 'mse',
        'smoothness': 0.000005,
        # 0.000005 gives 0.505823562521005
        # 0.00005 gives 0.5051972572416786
        # 0.0000005 gives 0.5043545818049401
        
        'smoothness_name': '0.000005',
        
        # with smoothness set to 0.00005,
        # scale=0.01 gives 0.505823562521005
        # scale=0.1 gives 0.48046038315427236
        # scale=0.001 gives 0.4979968643549016
        
        # my previous hyperparameters worked the best!!!
        
        'scale': 0.01,
        'scale_name': '0.01',
        'input_size': 40,
        'n_timesteps': 1,
    }
}

print(train_one_maskcnn_polished_with_rcnn_k_bl(**maskcnn_param_regular))

# 27629 = 1*2 + 16*1*9*9 + 16*2 + 2*(16*16*3*3+16*16*3*3+4*16*2) + 79*(14*14 + 16 + 1) 

{'final_act', 'pooling', 'fc', 'bn_output'}
['conv0', 'bl_stack.layer_list.0.b_conv', 'bl_stack.layer_list.1.b_conv']
neurons with NaN mean on train 230/345
num_param 53612


/my_data/thesis-yimeng-v2/thesis_v2/training_extra/maskcnn_like/training.py:122: RuntimeWarning: Mean of empty slice
  resp_mean = np.nanmean(resp_train, axis=0)


JSONNet(
  (moduledict): ModuleDict(
    (accumulator): RecurrentAccumulator()
    (act0): ReLU()
    (bl_stack): BLConvLayerStack(
      (layer_list): ModuleList(
        (0): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (1): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (bn_layer_list): ModuleList(
        (0): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (capture_list): ModuleList(
        (0): Identity()
        (1): Identity()
      )
      (input_capture): Identity()
      (act_fn): ReLU(inplace=True)
      (pool): Identity()
    )
    (bn0): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (bn_input): BatchNorm2d(1, eps=0.001, momentum

/opt/conda/envs/leelab/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/envs/leelab/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/opt/conda/envs/leelab/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


========starting epoch 100==========
100-0, train loss 0.29468151926994324
train loss 0.29468151926994324
val metric {'loss': 0.289869612455368, 'loss_no_reg': 0.28723782300949097, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12989900385676423, 0.37391398875679505, 0.37995191338941825, 0.44678947412748543, 0.47884927882479145, 0.37110948473740013, 0.37481473567337303, 0.4116197352838245, 0.2663329042840515, 0.14402324153463417, 0.4410373415842213, 0.11348

...

========starting epoch 400==========
400-0, train loss 0.2399551123380661
train loss 0.2399551123380661
val metric {'loss': 0.2621902525424957, 'loss_no_reg': 0.25590190291404724, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21769631168678524, 0.4754637709585046, 0.4467188817357695, 0.4658892111310281, 0.6106503489432528, 0.4857701476187599, 0.40792227442433, 0.4135840987467252, 0.3032383372023527, 0.15170742270980192, 0.4942000597646701, 0.25205173944958

early stopping after epoch 550 metric 0.2552691698074341
========end phase 2/3==========
========starting phase 3/3==========
for grp of sz 14, lr from 0.000333 to 0.000111
val metric init {'loss': 0.2613478899002075, 'loss_no_reg': 0.2552691698074341, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21638792546495286, 0.4743080893179375, 0.449609252430627, 0.46445744128836114, 0.6159404753311829, 0.48426799822661865, 0.4097039532109341, 0.413148012370167, 0.

========starting epoch 100==========
100-0, train loss 0.23467910289764404
train loss 0.23467910289764404
val metric {'loss': 0.2615516781806946, 'loss_no_reg': 0.2553827464580536, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21727846375918836, 0.4755220892996728, 0.4492411306728692, 0.46590976526587885, 0.6136761585045325, 0.4831731474626872, 0.4103510903337443, 0.4122001792759241, 0.2958297537479395, 0.15270816262910736, 0.4932767353984261, 0.2503236352

========starting epoch 300==========
300-0, train loss 0.24219219386577606
train loss 0.24219219386577606
val metric {'loss': 0.2617914706468582, 'loss_no_reg': 0.25559496879577637, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2166216877308387, 0.47597593463818183, 0.4489469908457894, 0.46559873820725767, 0.6130159016286556, 0.4837673143396099, 0.4107618847996932, 0.41145116913527696, 0.29851505452157795, 0.15171111405155166, 0.49326081765345026, 0.250380

early stopping after epoch 450 metric 0.2552691698074341
========end phase 3/3==========
(0.16885374961889824, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2389637231332898, 0.4751111918080948, 0.484340618999011, 0.46711598936762205, 0.6277211525431305, 0.4832223747674673, 0.3702225729934384, 0.428555300237799, 0.3227842153858949, 0.22082867487195035, 0.5082683616224101, 0.22611347811008856, 0.5466990923268454, 0.27851215628597265, 0.5491995710996486, 0.240569098

In [4]:
# get the non-zero part.

this_one = [0.2389637231332898, 0.4751111918080948, 0.484340618999011, 0.46711598936762205, 0.6277211525431305, 0.4832223747674673, 0.3702225729934384, 0.428555300237799, 0.3227842153858949, 0.22082867487195035, 0.5082683616224101, 0.22611347811008856, 0.5466990923268454, 0.27851215628597265, 0.5491995710996486, 0.24056909885416994, 0.3067944279503608, 0.18853489969042037, 0.3520387252482158, 0.47807766776573146, 0.3841453470343389, 0.573078043711403, 0.34181296379366666, 0.40688159687759734, 0.46569430805160117, 0.31056308766676843, 0.48738811420108086, 0.5167436115252766, 0.6373037845218286, 0.4343769439034203, 0.40988879279609586, 0.4462139741898665, 0.6393450887731075, 0.504812770721877, 0.48087317100252747, 0.5059954346960845, 0.35329032927067855, 0.5660956206842588, 0.5741532668752792, 0.11426053003050185, 0.3794082117390192, 0.515174647454766, 0.35913259143485765, 0.37839171867719734, 0.2894011988311822, 0.4039370218221169, 0.6557283065081818, 0.5130314522003574, 0.47406865875613124, 0.5453340685320515, 0.6851244854204872, 0.5601015098521744, 0.5825365003490603, 0.506589705574634, 0.6620267688927424, 0.6734501776020135, 0.7028090722084828, 0.4545743102999729, 0.745277851908858, 0.2993494770879035, 0.4674448548344973, 0.3996422067256073, 0.4692587347490887, 0.5249743062629934, 0.5165795674459063, 0.53332847786811, 0.5553527033186195, 0.37892866193274677, 0.5282501628730937, 0.6185608843818899, 0.540957470756707, 0.7476575181061355, 0.520291292872514, 0.40212938706682466, 0.5555566047096335, 0.47445466050045193, 0.6409082735731744, 0.5962676876798599, 0.17564080871527246, 0.5054441086729256, 0.5469809641928916, 0.5857018237185053, 0.6268002688940146, 0.7382219944798748, 0.6531805403930987, 0.7703910662493504, 0.6976815172164226, 0.6043149415510739, 0.4632691348399475, 0.5167628562997636, 0.7254362164558857, 0.6505843450274204, 0.43080287205483914, 0.5803649824492475, 0.5886477010169013, 0.6314078598443927, 0.5039800203355542, 0.7859944304998456, 0.5041495304767746, 0.541097255659085, 0.5463624550956401, 0.4629716124681924, 0.5792634655880918, 0.6852757517295472, 0.5774777234401135, 0.6836281839598343, 0.5984670376450211, 0.6096009480299416, 0.5607503334729095, 0.42320867139741347, 0.5527465340339877, 0.40385253109013963, 0.6617800705458726, 0.672021240610478, 0.6077005561707605]

In [5]:
# from rcnn_basic_kriegeskorte/test_full_training_ff_1st_cb19_data-handle_nan.ipynb

that_one = [0.22290867814140142, 0.4708074534173289, 0.4748005428071667, 0.4726406797165408, 0.6261681532403353, 0.47778410642306074, 0.355276401771913, 0.4059499988799383, 0.27306687854409467, 0.2060898428331426, 0.5066163009016792, 0.21927353933146942, 0.5601374651206583, 0.2686960626493656, 0.5464271834695443, 0.2654781651778954, 0.32077724062824814, 0.23326450810839897, 0.3647180023372379, 0.47421547330506253, 0.3785651481083519, 0.5456766079217238, 0.3364432552323184, 0.4076604394937908, 0.4603158349474313, 0.3521687998817147, 0.4759313387428043, 0.5159867024433585, 0.6517636553453058, 0.44227082061214457, 0.42572542630461446, 0.46016411728227063, 0.6354862858793571, 0.48589746898023056, 0.4898000842666996, 0.5060820441937934, 0.3344468595601764, 0.5575963223930174, 0.565608854549304, 0.14956376027860313, 0.3688827327773246, 0.5157210848584867, 0.37949105748215006, 0.38131425254857293, 0.29023178616260425, 0.4092873045632762, 0.6546056913526376, 0.5105307686644647, 0.4648774654180571, 0.5389574293108751, 0.6848201217249001, 0.5474474526035918, 0.5821566244878427, 0.51226782001166, 0.6609746645934759, 0.6702189332496112, 0.713059286114458, 0.45907239875451583, 0.7417166861688765, 0.30437573690915565, 0.4779997589486579, 0.427084702651328, 0.4953790066043401, 0.519572799621041, 0.5208000639149223, 0.5429269548689853, 0.5504635125870688, 0.3740523838454645, 0.5148547506060256, 0.617953049407335, 0.5446498963984046, 0.7483751444632495, 0.5351439216693811, 0.411975940704237, 0.5626833094802879, 0.48104878226225906, 0.6523571578566921, 0.6083369240852424, 0.15578991723678842, 0.5054916925086076, 0.5579364138230724, 0.5694297383452513, 0.6215502504670586, 0.7279874841137555, 0.6224736989941038, 0.761737156238404, 0.6861978108122229, 0.6067185997711673, 0.44793477697308465, 0.5357085593856187, 0.7331999385987633, 0.6475202233616871, 0.44000115178619764, 0.5841800693523288, 0.581523834811977, 0.6218072448705786, 0.5003692096558175, 0.7780975902298893, 0.49409784451955313, 0.527406207860786, 0.5361577988956837, 0.4588214757156862, 0.5553959701146822, 0.7058145972361495, 0.5886926663525995, 0.6853363853392981, 0.5815182730815066, 0.5858139427845256, 0.5553463898266813, 0.42168832978027504, 0.5526428668634553, 0.40211562266310247, 0.6608694233816872, 0.6747063506035611, 0.59971232260701]

In [6]:
from scipy.stats import pearsonr

In [7]:
# basically the same
pearsonr(this_one, that_one)

(0.9948890811382224, 2.250239568911009e-114)

In [8]:
np.mean(this_one), np.mean(that_one)

(0.5065612488566947, 0.505823562521005)

In [9]:
np.std(this_one), np.std(that_one)

(0.13644540535272634, 0.13470891707205376)